# Finery Image Recognition/Classification

This is the code for loading images, training an transfered image recognition model, testing on single images, and also testing on batch.

## Load Dependencies

We will need tensorflow, keras and numpy for this notebook

In [ ]:
import tensorflow as tf
import keras
import numpy as np

from keras import backend as K
from keras.preprocessing import image
from keras.applications import resnet50
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.layers import Dense,GlobalAveragePooling2D
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint

## Checking if GPU is recognized by tensorflow

if GPU present and recognized, it will show: `"/device:GPU:0"` in the output

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Limit GPU memory to 30% capacity per process, setting too high might hang the program, or may crash the machine!

In [ ]:
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

## Load the training and testing data using ImageDataGenerator

*If you made changes to the images' directory, here is the place to update them*

General structure of the images:

`./images/train/110/...jpg
    |           120
    |           130
    |           140
    |           150
    |           160
    |           ...
    ----->/test/110/...jpg
                120
                130
                140
                150
                160
                ...`

You can also specify the rotation of images (for improved feature extraction), and different preprocess to the images at this step.

In [ ]:
train_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input) # define a generator
train_generator = train_datagen.flow_from_directory('images/train/', # the training image file path
                                                 target_size=(224,224), # resnet takes input of size 244 by 244
                                                 color_mode='rgb', # top-less resnet takes input size of (3,244,244)
                                                 batch_size=32, # batch size for loading by generator
                                                 class_mode='categorical',
                                                 shuffle=True)

In [ ]:
test_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory('images/test/',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

These creates the training data as a generator class. From this you can implement preprocessing to the training and testing images, such as resizing, rotation, labeling, etc.

It is a good practice to check that both generator has the same number of categories.

## Load a "top-less" ResNet 50 model

This command will load a resnet 50 model without the final prediction layer, allowing us to append our own prediction layer at the end of the convolutional layer output.

In [ ]:
resnet_model = resnet50.ResNet50(weights='imagenet', include_top=False)

## Make a few layers of fully connected layer that attaches to resnet model output

The purpose of these layers is to serve as prediciton layers to train our image on.

In [ ]:
x = resnet_model.output # make a new layer that attaches to the resnet model output
x = GlobalAveragePooling2D()(x) # flattens the output using global average
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
final = Dense(12, activation='softmax')(x) # output layer, make sure the number
                                           # is same # of classes from above block

model=Model(inputs=resnet_model.input,outputs=final) # model takes input of resnet and output from our final layer,
                                                     # there is a function in keras that prints the layers of the
                                                     # model in a clean and concise way, have to check

## Check the model structure, Locate the newly created layers

For **ResNet**, layer 1 to 174 are supposed to be frozen and layers from 174 on are supposed to be set to trainable, depends on how many layers we created in the previous step.

In [ ]:
for i,layer in enumerate(model.layers):
  print(i,layer.name)

Also, the following code shows a pretty model structure, input, output size and number of parameters of the model

In [ ]:
model.summary()

## Set the convolutional layers not trainable, retaining their weights, Force the newly created layers to be trainable

In [ ]:
for layer in model.layers[:174]: # freeze layer 0 to 174
    layer.trainable=False

In [ ]:
for layer in model.layers[174:]: # make the layers we created trainable
    layer.trainable=True

## Defined Metrics:

We define these metrics so that we can use them at the callback stage for each epoch.

### AUC, area under curve, area under ROC curve

AUC is an appropriate measure of model accuracy for binary classifications. 

In [ ]:
def auc(y_true, y_pred):
     auc = tf.metrics.auc(y_true, y_pred)[1]
     K.get_session().run(tf.local_variables_initializer())
     return auc

### $F_1$ Score

$F_1$ score is the harmonic average of precision and recall, another metric for accuracy.

It is defined as: $F_1 = 2 \cdot \frac{\text{precision} \cdot \text{recall}}{\text{precision} + \text{recall}}$

In [ ]:
def f1(y_true, y_pred):
     f1 = tf.contrib.metrics.f1_score(y_true, y_pred)[1]
     K.get_session().run(tf.local_variables_initializer())
     return f1

### Top One Accuracy

This is a built-in metric for categorical classification in Keras. This is essentially the same as accuracy but we specify k=1 so that it calculates the micro-average accuracy.

In [ ]:
def top_1(x,y):
    return keras.metrics.top_k_categorical_accuracy(x,y,k=1)

### Top Three Accuracy

This is a built-in metric for categorical classification in Keras.

In [ ]:
def top_3(x,y):
    return keras.metrics.top_k_categorical_accuracy(x,y,k=3)

## Compile the model proper optimizers, loss function, and metric

More experiments can be done on this part, we are just choosing the optimizers, loss and metrics that, through testing, makes the most sense here.

In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=[top_1,top_3])

## Calculate penalizing weights using negative frequencies.

Negative frequencies $W_c$ is defined as the log smoothed ratio of the total number of samples $N$ to number of samples in specific category $n_c$:

$W_c = \log \left( \frac{N}{n_c} \right) \text{if } > 1 \text{, else }1$.

Which means if this smoothed ratio is less than 1, the weight for that particular category will be 1, else being the calculated ratio.

In [ ]:
cw ={0:1, 1:1, 2:1.9659, 3:1.168, 4:1.339, 5:1.3629, 6:2.3743, 7:1, 8:1, 9:1.0239, 10:1, 11:1.3792}

## Define callback precedures after each epoch end

Implement early stopping by using the validation set's loss as a stopping metric, `patiance=2` means stopping after two epoch if the loss does not decrease. This is quite strict and might require tuning when sample size is large or undersampling is used.

In [ ]:
early = EarlyStopping(monitor="val_loss", mode="min", patience=2)

Implement checkpoint by monitoring the validation set's loss, only saves the best model achieve to avoid occupying the hard drive too much.

This callback also creates the model structure and weights file in `*.hdf5`, we can load this weight directly without training again later.

In [ ]:
checkpoint = ModelCheckpoint(filepath='best.hdf5', monitor='val_loss', save_best_only=True)

## Define the batch size of training

Here we use the total number of samples divide by the batch size of the image generator. Change this when sample size is large, otherwise will hang.

In [ ]:
step_size_train=train_generator.n//train_generator.batch_size
step_size_test=test_generator.n//test_generator.batch_size

## Fit the image data on the model using GPU

Here we use the test set as the validation set. Not only do we see test accuracy after every epoch, we also can have early stopping so the model won't overfit. Use `epoch=20` since we want the model to be fully trained without stopping too early. However from our experience, the training usually stops at epoch 4 to 5.

In [ ]:
with tf.device('/gpu:0'):
    model.fit_generator(generator=train_generator,
                        validation_data=test_generator,
                        steps_per_epoch=step_size_train,
                        validation_steps=step_size_test,
                        epochs=20,
                        class_weight=cw,
                        callbacks=[early, checkpoint])

## To see accuracy of test data in batch

There will be several outputs, the first will always be loss, the next few will be the metric you defined in order.

In [ ]:
model.evaluate_generator(generator=test_generator, steps=step_size_test)

$\space$

$\space$

$\space$

$\space$

$\space$

$\space$

$\space$

# Model using VGG16:

Same structure just different conv. layer

In [ ]:
from keras.applications import vgg16
vgg_model = vgg16.VGG16(weights='imagenet', include_top=False)

x = vgg_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
final = Dense(12, activation='softmax')(x)
model2=Model(inputs=vgg_model.input,outputs=final)

for layer in model2.layers[:18]:
    layer.trainable=False
for layer in model2.layers[18:]:
    layer.trainable=True
    
model2.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=[top_1,top_3])

checkpoint2 = ModelCheckpoint(filepath='best_vgg.hdf5', monitor='val_loss', save_best_only=True)

with tf.device('/gpu:0'):
    model2.fit_generator(generator=train_generator,
                         validation_data=test_generator,
                         steps_per_epoch=step_size_train,
                         validation_steps=step_size_test,
                         epochs=20,
                         class_weight=cw,
                         callbacks=[early, checkpoint2])

$\space$

$\space$

$\space$

$\space$

$\space$

$\space$

$\space$

# Binary classification case >>misleading results<<

Images in train and test will be put in `non-wardrobe` and `wardrove` folders.

In [ ]:
train_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory('hot_dog/train/',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)
test_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory('hot_dog/test/',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

In [ ]:
x = resnet_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
final = Dense(2, activation='softmax')(x)
model3=Model(inputs=resnet_model.input,outputs=final)

for layer in model3.layers[:174]:
    layer.trainable=False
for layer in model3.layers[174:]:
    layer.trainable=True
    
model3.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=[auc,f1])

cw ={0:1.3792, 1:1}

checkpoint_hotdog = ModelCheckpoint(filepath='best_hotdog.hdf5', monitor='val_loss', save_best_only=True)

step_size_train = train_generator.n//train_generator.batch_size
step_size_test = test_generator.n//test_generator.batch_size

with tf.device('/gpu:0'):
    model.fit_generator(generator=train_generator,
                        validation_data=test_generator,
                        steps_per_epoch=step_size_train,
                        validation_steps=step_size_test,
                        epochs=20,
                        class_weight=cw,
                        callbacks=[early, checkpoint_hotdog])

$\space$

$\space$

$\space$

$\space$

$\space$

$\space$

$\space$

# Classifying on all 57 categories

In [ ]:
train_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory('images/train/', 
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

test_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory('images/test/',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

In [ ]:
x = resnet_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
final = Dense(57, activation='softmax')(x)
model = Model(inputs=resnet_model.input,outputs=final)

for layer in model5.layers[:174]:
    layer.trainable=False
for layer in model5.layers[174:]:
    layer.trainable=True
    
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=[top_1,top_3])

checkpoint5 = ModelCheckpoint(filepath='best_resnet_cat.hdf5', monitor='val_loss', save_best_only=True)

cw = {0:1.50131362, 1:1.1820588, 2:1.53955363, 3:1.74326729, 4:1.48907916,
      5:1.80859256, 6:1.81749496, 7:1.88701917, 8:1.63040832, 9:1.53752579,
      10:2.0139732, 11:3.06765806, 12:2.33781149, 13:2.23246831, 14:2.24604111,
      15:1.20684739, 16:3.02419236, 17:2.82789772, 18:3.11596274, 19:2.06311043,
      20:2.14501453, 21:1.74543876, 22:2.3129879, 23:2.79010916, 24:2.11852495,
      25:1.69416532, 26:2.18504366, 27:3.34641166, 28:1.75870069, 29:3.26723041,
      30:3.8692904, 31:3.50131362, 32:2.38737027, 33:1.38594088, 34:1.30108868,
      35:1.71700206, 36:1.5837331, 37:1.61161183, 38:1.5807431 , 39:1.78531028,
      40:1.85786094, 41:1.19259679, 42:2.05415559, 43:1.99422914, 44:1.75091544,
      45:2.00995193, 46:1.2827031, 47:2.13958578, 48:2.05194543, 49:2.0346578,
      50:2.85504997, 51:2.1703204, 52:2.65621558, 53:2.89925363, 54:1,
      55:3.06765806, 56:1.36008788}

step_size_train = train_generator.n//train_generator.batch_size
step_size_test = test_generator.n//test_generator.batch_size

with tf.device('/gpu:0'):
    model.fit_generator(generator=train_generator,
                        validation_data=test_generator,
                        steps_per_epoch=step_size_train,
                        validation_steps=step_size_test,
                        epochs=100,
                        class_weight=cw,
                        callbacks=[early, checkpoint5])